In [1]:
######################
## Minimum Variance ##
######################

## Importing yahoo finance 
import yfinance as yf

## Downloading Amazon stock price data 
amazon = yf.download('AMZN', start = '2020-01-01', end = '2020-12-31')

## Downloading Apple stock price data 
apple = yf.download('AAPL', start = '2020-01-01', end = '2020-12-31')

## Importing pandas and numpy 
import pandas as pd
import numpy as np

## Computing the total returns
returns = pd.DataFrame({'AMZN_tot_return': amazon['Adj Close'].pct_change(), 
                        'AAPL_tot_return': apple['Adj Close'].pct_change()})
returns = returns.dropna()

## Creating a series of weights
weights = pd.DataFrame({'AMZN_wgt': np.arange(0, 1, 0.01)})
weights['AAPL_wgt'] = 1 - weights['AMZN_wgt']

## Computing the variance and the covariance
var_amazon = returns['AMZN_tot_return'].var()
var_apple = returns['AAPL_tot_return'].var()
cov_amazon_apple = returns[['AMZN_tot_return', 'AAPL_tot_return']].cov().iloc[0, 1]

## Computing the variance of each weight combination
weights['portfolio_variance'] = weights['AMZN_wgt']**2 * var_amazon + weights['AAPL_wgt']**2 * var_apple + 2*cov_amazon_apple*weights['AMZN_wgt']*weights['AAPL_wgt']

## Identifying portfolio with minimum variance
weights_min = weights.loc[weights['portfolio_variance'].idxmin(), ]
weights_min

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AMZN_wgt              0.800000
AAPL_wgt              0.200000
portfolio_variance    0.000572
Name: 80, dtype: float64

In [2]:
##########################
## Maximum Sharpe Ratio ##
##########################

## Importing yahoo finance 
import yfinance as yf

## Downloading Amazon stock price data 
amazon = yf.download('AMZN', start = '2020-01-01', end = '2020-12-31')

## Downloading Apple stock price data 
apple = yf.download('AAPL', start = '2020-01-01', end = '2020-12-31')

## Importing pandas and numpy 
import pandas as pd
import numpy as np

## Computing the total returns
returns = pd.DataFrame({'AMZN_tot_return': amazon['Adj Close'].pct_change(), 
                        'AAPL_tot_return': apple['Adj Close'].pct_change()})
returns = returns.dropna()

## Creating a series of weights
weights = pd.DataFrame({'AMZN_wgt': np.arange(0, 1, 0.01)})
weights['AAPL_wgt'] = 1 - weights['AMZN_wgt']

## Computing the portfolio returns for each weight combination
weights['portfolio_return'] = weights['AMZN_wgt']*returns['AMZN_tot_return'].mean() + weights['AAPL_wgt']*returns['AAPL_tot_return'].mean()

## Computing the variance and the covariance
var_amazon = returns['AMZN_tot_return'].var()
var_apple = returns['AAPL_tot_return'].var()
cov_amazon_apple = returns[['AMZN_tot_return', 'AAPL_tot_return']].cov().iloc[0, 1]

## Computing the variance of each weight combination
weights['portfolio_variance'] = weights['AMZN_wgt']**2 * var_amazon + weights['AAPL_wgt']**2 * var_apple + 2*cov_amazon_apple*weights['AMZN_wgt']*weights['AAPL_wgt']

## Computing the Sharpe Ratio of each weight combination
daily_risk_rate = 0.0008 / 365
weights['Sharpe_Ratio'] = (weights['portfolio_return'] - daily_risk_rate) / np.sqrt(weights['portfolio_variance'])

## Identifying portfolio with minimum variance
weights_max = weights.loc[weights['Sharpe_Ratio'].idxmax(), ]
weights_max

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AMZN_wgt              0.660000
AAPL_wgt              0.340000
portfolio_return      0.002580
portfolio_variance    0.000582
Sharpe_Ratio          0.106880
Name: 66, dtype: float64

In [3]:
##############################################
## Minimum Variance (quadratic programming) ##
##############################################

## Importing yahoo finance 
import yfinance as yf

## Downloading Amazon stock price data 
amazon = yf.download('AMZN', start = '2020-01-01', end = '2020-12-31')

## Downloading Apple stock price data 
apple = yf.download('AAPL', start = '2020-01-01', end = '2020-12-31')

## Importing pandas and numpy 
import pandas as pd
import numpy as np

## Computing the total returns
returns = pd.DataFrame({'AMZN_tot_return': amazon['Adj Close'].pct_change(), 
                        'AAPL_tot_return': apple['Adj Close'].pct_change()})
returns = returns.dropna()

## Importing scipy
from scipy.optimize import minimize 

## Defining the portfolio variance
def minimum_variance(x, cov_matrix):
    f = float(np.sqrt(x.dot(cov_matrix).dot(x.T)))
    return f 

## Defining the number of stocks
n = returns.shape[1]

## Defining the initial weights
x = np.repeat(1/n, n)

## Defining the covariance matrix
cov_matrix = returns.cov()

## Defining the matrix of contraints
cons = ({'type': 'eq','fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1,) for i in range(len(x))]

## Solving the optimization problem
result_mv = minimize(minimum_variance, x, args = (cov_matrix), method = 'SLSQP', bounds = bounds, constraints = cons).x 
result_mv

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


array([0.80162454, 0.19837546])

In [4]:
##################################################
## Maximum Sharpe-Ratio (quadratic programming) ##
##################################################

## Importing yahoo finance 
import yfinance as yf

## Downloading Amazon stock price data 
amazon = yf.download('AMZN', start = '2020-01-01', end = '2020-12-31')

## Downloading Apple stock price data 
apple = yf.download('AAPL', start = '2020-01-01', end = '2020-12-31')

## Importing pandas and numpy 
import pandas as pd
import numpy as np

## Computing the total returns
returns = pd.DataFrame({'AMZN_tot_return': amazon['Adj Close'].pct_change(), 
                        'AAPL_tot_return': apple['Adj Close'].pct_change()})
returns = returns.dropna()

## Importing scipy
from scipy.optimize import minimize 

## Defining the portfolio Sharpe Ratio
def sharpe_ratio(x, cov_matrix, mean_vector, r_f):
    f = float(-(x.dot(mean_vector) - r_f) / np.sqrt(x.dot(cov_matrix).dot(x.T)))
    return f 

## Defining the risk free rate
daily_risk = 0.0008 / 365

## Defining the initial weights
x = np.repeat(1/n, n)

## Defining the mean returns
mean_vector = np.mean(returns, axis=0)

## Defining the covariance matrix
cov_matrix = returns.cov()

## Defining the matrix of contraints
cons = ({'type': 'eq','fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1,) for i in range(len(x))]

## Solving the optimization problem
result_Sharpe = minimize(sharpe_ratio, x, args=(cov_matrix, mean_vector, daily_risk), method = 'SLSQP', bounds = bounds, constraints = cons).x 
result_Sharpe

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


array([0.65901238, 0.34098762])

In [5]:
##############################################
## Minimum Variance (quadratic programming) ##
##############################################

## Importing yahoo finance 
import yfinance as yf

## Downloading Amazon stock price data 
amazon = yf.download('AMZN', start = '2020-01-01', end = '2020-12-31')

## Downloading Apple stock price data 
apple = yf.download('AAPL', start = '2020-01-01', end = '2020-12-31')

## Downloading Netflix stock price data
netflix = yf.download('NFLX', start = '2020-01-01', end = '2020-12-31')

## Downloading Tesla stock price data
tesla = yf.download('TSLA', start = '2020-01-01', end = '2020-12-31')

## Downloading Google stock price data
google = yf.download('GOOG', start = '2020-01-01', end = '2020-12-31')

## Importing pandas and numpy 
import pandas as pd
import numpy as np

## Computing the total returns
returns = pd.DataFrame({'AMZN_tot_return': amazon['Adj Close'].pct_change(), 
                        'AAPL_tot_return': apple['Adj Close'].pct_change(),
                        'NFLX_tot_return': netflix['Adj Close'].pct_change(),
                        'TSLA_tot_return': tesla['Adj Close'].pct_change(),
                        'GOOG_tot_return': google['Adj Close'].pct_change()})
returns = returns.dropna()

## Importing scipy
from scipy.optimize import minimize 

## Defining the portfolio variance
def minimum_variance(x, cov_matrix):
    f = float(np.sqrt(x.dot(cov_matrix).dot(x.T)))
    return f 

## Defining the number of stocks
n = returns.shape[1]

## Defining the initial weights
x = np.repeat(1/n, n)

## Defining the covariance matrix
cov_matrix = returns.cov()

## Defining the matrix of contraints
cons = ({'type': 'eq','fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1,) for i in range(len(x))]

## Solving the optimization problem
result_mv = minimize(minimum_variance, x, args = (cov_matrix), method = 'SLSQP', bounds = bounds, constraints = cons).x 
result_mv

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


array([3.74303592e-01, 0.00000000e+00, 1.78998270e-01, 1.04083409e-17,
       4.46698139e-01])

In [6]:
##################################################
## Maximum Sharpe-Ratio (quadratic programming) ##
##################################################

## Importing yahoo finance 
import yfinance as yf

## Downloading Amazon stock price data 
amazon = yf.download('AMZN', start = '2020-01-01', end = '2020-12-31')

## Downloading Apple stock price data 
apple = yf.download('AAPL', start = '2020-01-01', end = '2020-12-31')

## Downloading Netflix stock price data
netflix = yf.download('NFLX', start = '2020-01-01', end = '2020-12-31')

## Downloading Tesla stock price data
tesla = yf.download('TSLA', start = '2020-01-01', end = '2020-12-31')

## Downloading Google stock price data
google = yf.download('GOOG', start = '2020-01-01', end = '2020-12-31')

## Importing pandas and numpy 
import pandas as pd
import numpy as np

## Computing the total returns
returns = pd.DataFrame({'AMZN_tot_return': amazon['Adj Close'].pct_change(), 
                        'AAPL_tot_return': apple['Adj Close'].pct_change(),
                        'NFLX_tot_return': netflix['Adj Close'].pct_change(),
                        'TSLA_tot_return': tesla['Adj Close'].pct_change(),
                        'GOOG_tot_return': google['Adj Close'].pct_change()})
returns = returns.dropna()

## Importing scipy
from scipy.optimize import minimize 

## Defining the portfolio Sharpe Ratio
def sharpe_ratio(x, cov_matrix, mean_vector, r_f):
    f = float(-(x.dot(mean_vector) - r_f) / np.sqrt(x.dot(cov_matrix).dot(x.T)))
    return f 

## Defining the risk free rate
daily_risk = 0.0008 / 365

## Defining the initial weights
x = np.repeat(1/n, n)

## Defining the mean returns
mean_vector = np.mean(returns, axis=0)

## Defining the covariance matrix
cov_matrix = returns.cov()

## Defining the matrix of contraints
cons = ({'type': 'eq','fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1,) for i in range(len(x))]

## Solving the optimization problem
result_Sharpe = minimize(sharpe_ratio, x, args=(cov_matrix, mean_vector, daily_risk), method = 'SLSQP', bounds = bounds, constraints = cons).x 
result_Sharpe

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


array([2.75056736e-01, 0.00000000e+00, 5.42101086e-18, 7.24943264e-01,
       0.00000000e+00])